In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import sys
import glob
sys.path.append('..')
from preprocessing import extract_bike_stations, clean_stations, concat_bike_data, clean_trip_data,\
fetch_weather_data, concat_weather_data, clean_weather_data, clean_wards,clean_accidents

## Create Generic city csv

In [8]:
# Create dataframe
df_cities = pd.DataFrame({"city_id": [0], "city_name": ["Toronto"]})
df_cities.to_csv('../data/processed/cities.csv')

## Clean Wards

In [4]:
# Read wards 
df_wards = pd.read_csv('../data/raw/25 Ward Names and Numbers.csv')
df_wards_geo = gpd.read_file('../data/raw/City Wards Data.geojson')

# Clean Wards
df_wards, df_wards_geo = clean_wards(df_wards, df_wards_geo, save=True)

/Users/kh/PycharmProjects/pythonProject2/venv/lib/python3.9/site-packages/geopandas/io/file.py:383: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  as_dt = pd.to_datetime(df[k], errors="ignore")
/Users/kh/PycharmProjects/pythonProject2/venv/lib/python3.9/site-packages/geopandas/io/file.py:387: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  as_dt = pd.to_datetime(df[k], errors="ignore", utc=True)


## Clean Neighborhood Data - NOT USED ANYMORE

In [ ]:
# # Load neighborhood data
# neighborhoods_gdf = gpd.read_file('/content/Neighbourhoods - historical 140.geojson')

# Clean neighborhood geodata
# neighborhoods_gdf = clean_neighbors(neighborhoods_gdf)

#save geodata as csv
# neighborhoods_gdf.to_csv('../data/processed/toronto_neighborhoods.csv')

In [5]:
# df_neighbors = pd.read_csv('../data/processed/toronto_neighborhoods.csv', index_col = 0)
# df_neighbors

,area_id,city_id,neighborhood_id,area_name,objectid,geometry
0,2501286,0,30,Brookhaven-Amesbury,17818369.0,MULTIPOLYGON (((-79.5029569066111 43.695737091...
1,2501285,0,106,Humewood-Cedarvale,17818385.0,MULTIPOLYGON (((-79.4184870633337 43.683629484...
2,2501284,0,91,Weston-Pelham Park,17818401.0,MULTIPOLYGON (((-79.4600458827451 43.667225962...
3,2501283,0,48,Hillcrest Village,17818417.0,MULTIPOLYGON (((-79.3434566072996 43.795165708...
4,2501282,0,129,Agincourt North,17818433.0,MULTIPOLYGON (((-79.2421274578355 43.802472005...
...,...,...,...,...,...,...
135,2501151,0,20,Alderwood,17820529.0,MULTIPOLYGON (((-79.5486574524729 43.590216110...
136,2501150,0,21,Humber Summit,17820545.0,MULTIPOLYGON (((-79.5806438818039 43.763017918...
137,2501149,0,102,Forest Hill North,17820561.0,MULTIPOLYGON (((-79.4255619572768 43.700992532...
138,2501148,0,8,Humber Heights-Westmount,17820577.0,MULTIPOLYGON (((-79.5132891036989 43.693383995...


## Clean Ride Data

In [3]:
# concat all trips from 2017 - 2022
# df_trips = concat_bike_data()

# clean trip data
# df_trips = clean_trip_data(df_trips, save = True)

# Read trip data
df_trips = pd.read_csv('../data/processed/bikeshare_trips.csv')
df_trips.head()

,trip_id,user_type,start_station_id,start_time,end_station_id,bike_id,duration
0,20148783,Casual Member,7027,2022-12-31 23:59:00,7269,113.0,14.0
1,20148782,Casual Member,7130,2022-12-31 23:59:00,7012,3182.0,10.0
2,20148777,Casual Member,7713,2022-12-31 23:58:00,7028,6306.0,8.0
3,20148776,Casual Member,7029,2022-12-31 23:58:00,7164,536.0,29.0
4,20148775,Casual Member,7374,2022-12-31 23:58:00,7048,788.0,47.0


In [11]:
df_trips_small = df_trips.sample(frac=0.08)

In [13]:
df_trips_small.to_csv('..data/process/bikeshare_trips_sampled.csv', index=False)

OSError: Cannot save file into a non-existent directory: '..data/process'

## Extract and Clean Station Data

In [4]:
# Scrape and save raw bike station data
# extract_bike_stations()

In [5]:
# # Open bike share data
df_stations = pd.read_csv('../data/raw/bikeshare_stations.csv')

# # Clean and one hot encode rental methods column
clean_stations(df_stations, df_trips, df_wards_geo)

/Users/kh/PycharmProjects/pythonProject2/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
../preprocessing.py:373: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  stations_with_neighborhoods = gpd.sjoin(stations_gdf, neighborhoods_gdf, how='left', op='within')
../preprocessing.py:391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations.first_use[pd.isna(df_stations.start_time_x)] = df_stations.start_time_y[pd.isna(df_sta

In [33]:
df_stations = pd.read_csv('../data/processed/bikeshare_stations.csv')
df_stations.head()

,station_id,name,lat,lon,address,capacity,ward_id,first_use
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,Fort York Blvd / Capreol Ct,35,10,2018-01-01
1,7002,St. George St / Bloor St W,43.667333,-79.399429,St. George St / Bloor St W,19,11,2018-01-01
2,7003,Madison Ave / Bloor St W,43.667158,-79.402761,Madison Ave / Bloor St W,15,11,2018-01-01
3,7004,University Ave / Elm St,43.656518,-79.389099,University Ave / Elm St,11,11,2018-01-01
4,7005,King St W / York St,43.648001,-79.383177,King St W / York St,23,10,2018-01-02


In [35]:
df_stations[df_stations.station_id == 7078]

,station_id,name,lat,lon,address,capacity,ward_id,first_use
69,7078,College St / Major St,43.6576,-79.4032,College St / Major St,15,11,2018-01-01


## Clean weather data

In [3]:
# # fetch weather data 
# fetch_weather_data()

# concat weather data
# df_weather = concat_weather_data()
# df_weather.to_csv('../data/raw/bike-share-weather.csv')

df_weather = pd.read_csv('../data/raw/bike-share-weather.csv', index_col = 0)

In [4]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2191 entries, 0 to 2190
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date/Time               2191 non-null   object 
 1   Mean Temp (°C)          2171 non-null   float64
 2   Total Rain (mm)         2175 non-null   float64
 3   Total Snow (cm)         2177 non-null   float64
 4   Spd of Max Gust (km/h)  1926 non-null   object 
dtypes: float64(3), object(2)
memory usage: 102.7+ KB


In [5]:
# clean weather data
df_weather = clean_weather_data(df_weather)

../preprocessing.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather.max_gust[df_weather.max_gust == '<31'] = '29'
../preprocessing.py:254: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather.gust_type[(df_weather.gust_type.isna()) & (df_weather.max_gust < 30)] = 'Moderate'


In [6]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2191 entries, 0 to 2190
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        2191 non-null   object 
 1   mean_temp   2191 non-null   float64
 2   total_rain  2191 non-null   float64
 3   total_snow  2191 non-null   float64
 4   max_gust    1926 non-null   float64
 5   gust_type   1926 non-null   object 
 6   city_id     2191 non-null   int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 136.9+ KB


In [7]:
df_weather.to_csv('../data/processed/bike-share_weather.csv')

## Clean accident data

In [4]:
# # Read raw accidents data
# df_accidents = pd.read_csv('../data/raw/cyclist-accidents.csv')

# # Clean accidents data
# df_accidents = clean_accidents(df_accidents, df_wards_geo)

# #Save cleaned data
df_accidents.to_csv('../data/processed/toronto_accidents.csv', index=False)

# df_accidents = pd.read_csv('../data/processed/toronto_accidents.csv', index_col = 0)

df_accidents.head()

,date,latitude,longitude,light,acclass,invtype,injury,cyclistype,cycact,geometry,ward_id
1000,2017-12-20,43.654326,-79.422664,Daylight,Fatal,Truck Driver,Fatal,NaN,NaN,POINT (-79.42266 43.65433),11
1001,2018-01-22,43.790740,-79.452991,Daylight,Non-Fatal Injury,Cyclist,Major,Motorist turned left across cyclists path.,Driving Properly,POINT (-79.45299 43.79074),6
1002,2018-01-22,43.790740,-79.452991,Daylight,Non-Fatal Injury,Driver,NaN,NaN,NaN,POINT (-79.45299 43.79074),6
1003,2018-01-22,43.701487,-79.365566,Daylight,Non-Fatal Injury,Driver,NaN,NaN,NaN,POINT (-79.36557 43.70149),15
1004,2018-01-22,43.701487,-79.365566,Daylight,Non-Fatal Injury,Cyclist,Major,Insufficient information (to determine cyclist...,Driving Properly,POINT (-79.36557 43.70149),15


## Clean population data - NOT USED ANYMORE

In [38]:
# # read raw data 
# df_pop = pd.read_csv('../data/raw/wellbeing_toronto.csv')

# # clean data
# df_pop = clean_pop(df_pop)
# df_pop.to_csv('../data/processed/toronto_population.csv')

df_pop = pd.read_csv('../data/processed/toronto_population.csv',index_col = 0)
df_pop.head()

,neighborhood_id,total_population,year
0,1,33312.0,2016
1,2,32954.0,2016
2,3,10360.0,2016
3,4,10529.0,2016
4,5,9456.0,2016
